In [1]:
# Basic imports
import pandas as pd
import numpy as np

In [2]:
# Imports and paths
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set paths. Raw CSVs are in the workspace Data/ folder.
BASE = Path('d:/slooze_challenge')
RAW_DATA = BASE / 'Data'
CLEANED_DIR = Path('d:/slooze_challenge/Slooze_Analysis/data/cleaned')
CLEANED_DIR.mkdir(parents=True, exist_ok=True)

print("Reading raw CSV files from:", RAW_DATA)
print("Cleaned files will be written to:", CLEANED_DIR)


Reading raw CSV files from: d:\slooze_challenge\Data
Cleaned files will be written to: d:\slooze_challenge\Slooze_Analysis\data\cleaned


In [3]:
# Expected filenames (these are the files you provided)
expected = {
    'sales': 'SalesFINAL12312016.csv',
    'purchases': 'PurchasesFINAL12312016.csv',
    'invoice_purchases': 'InvoicePurchases12312016.csv',
    'beg_inv': 'BegInvFINAL12312016.csv',
    'end_inv': 'EndInvFINAL12312016.csv',
    'prices': '2017PurchasePricesDec.csv'
}

loaded = {}
for key, fname in expected.items():
    path = RAW_DATA / fname
    if path.exists():
        try:
            df = pd.read_csv(path, low_memory=False)
            loaded[key] = df
            print(f"Read '{fname}' as '{key}'. Shape: {df.shape}")
        except Exception as e:
            print(f"Could not read '{fname}': {e}")
    else:
        print(f"File not found: {fname} (expected in {RAW_DATA})")


Read 'SalesFINAL12312016.csv' as 'sales'. Shape: (1048575, 14)
Read 'PurchasesFINAL12312016.csv' as 'purchases'. Shape: (2372474, 16)
Read 'InvoicePurchases12312016.csv' as 'invoice_purchases'. Shape: (5543, 10)
Read 'BegInvFINAL12312016.csv' as 'beg_inv'. Shape: (206529, 9)
Read 'EndInvFINAL12312016.csv' as 'end_inv'. Shape: (224489, 9)
Read '2017PurchasePricesDec.csv' as 'prices'. Shape: (12261, 9)


In [4]:
# Quick look at each loaded file (show first 2 rows and the column names)
def show_info(df, name):
    print(f"\n--- {name} (rows: {len(df)}) ---")
    print('Columns:', ', '.join(list(df.columns)))
    try:
        print(df.head(2).to_string(index=False))
    except Exception:
        print(df.head(2))

for k, df in loaded.items():
    show_info(df, k)



--- sales (rows: 1048575) ---
Columns: InventoryId, Store, Brand, Description, Size, SalesQuantity, SalesDollars, SalesPrice, SalesDate, Volume, Classification, ExciseTax, VendorNo, VendorName
        InventoryId  Store  Brand                Description  Size  SalesQuantity  SalesDollars  SalesPrice SalesDate  Volume  Classification  ExciseTax  VendorNo                  VendorName
1_HARDERSFIELD_1004      1   1004 Jim Beam w/2 Rocks Glasses 750mL              1         16.49       16.49  1/1/2016     750               1       0.79     12546 JIM BEAM BRANDS COMPANY    
1_HARDERSFIELD_1004      1   1004 Jim Beam w/2 Rocks Glasses 750mL              2         32.98       16.49  1/2/2016     750               1       1.57     12546 JIM BEAM BRANDS COMPANY    

--- purchases (rows: 2372474) ---
Columns: InventoryId, Store, Brand, Description, Size, VendorNumber, VendorName, PONumber, PODate, ReceivingDate, InvoiceDate, PayDate, PurchasePrice, Quantity, Dollars, Classification
      Invento

In [5]:
# Clean the sales file: standardize key columns and save a cleaned CSV
sales = loaded.get('sales')
if sales is not None:
    # Normalize column names
    sales.columns = [str(c).strip() for c in sales.columns]

    # Try to find a date column
    date_cols = [c for c in sales.columns if 'date' in c.lower() or 'invdate' in c.lower()]
    if date_cols:
        sales['date'] = pd.to_datetime(sales[date_cols[0]], errors='coerce')
    else:
        sales['date'] = pd.NaT

    # Find SKU / product column
    sku_cols = [c for c in sales.columns if any(k in c.lower() for k in ('sku','item','upc','product'))]
    qty_cols = [c for c in sales.columns if any(k in c.lower() for k in ('qty','quantity'))]
    price_cols = [c for c in sales.columns if any(k in c.lower() for k in ('price','amount','revenue','ext'))]

    sales['sku'] = sales[sku_cols[0]] if sku_cols else sales.iloc[:,0]
    sales['quantity'] = pd.to_numeric(sales[qty_cols[0]], errors='coerce') if qty_cols else 1

    if price_cols:
        sales['revenue'] = pd.to_numeric(sales[price_cols[0]], errors='coerce')
    else:
        sales['revenue'] = sales['quantity'] * 0

    sales = sales.drop_duplicates().reset_index(drop=True)
    out_path = CLEANED_DIR / 'sales_cleaned.csv'
    sales.to_csv(out_path, index=False)
    print(f"Saved cleaned sales to: {out_path}")
else:
    print('Sales file was not loaded — check Data/ folder for SalesFINAL12312016.csv')


Saved cleaned sales to: d:\slooze_challenge\Slooze_Analysis\data\cleaned\sales_cleaned.csv


In [6]:
# Clean purchases and prices, and save cleaned versions for later use
purchases = loaded.get('purchases')
prices = loaded.get('prices')

if purchases is not None:
    purchases.columns = [str(c).strip() for c in purchases.columns]
    date_cols = [c for c in purchases.columns if 'date' in c.lower()]
    if date_cols:
        purchases['purchase_date'] = pd.to_datetime(purchases[date_cols[0]], errors='coerce')
    purchases.to_csv(CLEANED_DIR / 'purchases_cleaned.csv', index=False)
    print('Saved cleaned purchases to:', (CLEANED_DIR / 'purchases_cleaned.csv'))
else:
    print('Purchases file not found or could not be read.')

if prices is not None:
    prices.columns = [str(c).strip() for c in prices.columns]
    prices.to_csv(CLEANED_DIR / 'prices_cleaned.csv', index=False)
    print('Saved cleaned prices to:', (CLEANED_DIR / 'prices_cleaned.csv'))
else:
    print('Prices file not found or could not be read.')

# Also save beginning and ending inventory files if present
beg_inv = loaded.get('beg_inv')
end_inv = loaded.get('end_inv')
if beg_inv is not None:
    beg_inv.to_csv(CLEANED_DIR / 'beginning_inventory_cleaned.csv', index=False)
    print('Saved beginning inventory to:', (CLEANED_DIR / 'beginning_inventory_cleaned.csv'))
if end_inv is not None:
    end_inv.to_csv(CLEANED_DIR / 'ending_inventory_cleaned.csv', index=False)
    print('Saved ending inventory to:', (CLEANED_DIR / 'ending_inventory_cleaned.csv'))

# Invoice purchases (if present) may contain order/receive dates useful for lead time
invoice = loaded.get('invoice_purchases')
if invoice is not None:
    invoice.to_csv(CLEANED_DIR / 'invoice_purchases_cleaned.csv', index=False)
    print('Saved invoice purchases to:', (CLEANED_DIR / 'invoice_purchases_cleaned.csv'))


Saved cleaned purchases to: d:\slooze_challenge\Slooze_Analysis\data\cleaned\purchases_cleaned.csv
Saved cleaned prices to: d:\slooze_challenge\Slooze_Analysis\data\cleaned\prices_cleaned.csv
Saved beginning inventory to: d:\slooze_challenge\Slooze_Analysis\data\cleaned\beginning_inventory_cleaned.csv
Saved ending inventory to: d:\slooze_challenge\Slooze_Analysis\data\cleaned\ending_inventory_cleaned.csv
Saved invoice purchases to: d:\slooze_challenge\Slooze_Analysis\data\cleaned\invoice_purchases_cleaned.csv


## Output
The cleaned CSVs are saved to the `Slooze_Analysis/data/cleaned/` folder.
Next notebook will load these cleaned files for analysis.
